<center>
<img src="../../img/ods_stickers.jpg">
## Открытый курс по машинному обучению
</center>
<center>Автор материала: программист-исследователь Mail.ru Group, старший преподаватель <br>Факультета Компьютерных Наук ВШЭ Юрий Кашницкий

# <center> Домашнее задание № 8. Часть 2
## <center> Vowpal Wabbit в задаче классификации тегов вопросов на Stackoverflow

## План 2 части домашнего задания
    2.1. Введение
    2.2. Описание данных
    2.3. Предобработка данных
    2.4. Обучение и проверка моделей
    2.5. Заключение

### 2.1. Введение

В этом задании вы будете делать примерно то же, что я каждую неделю –  в Mail.ru Group: обучать модели на выборке в несколько гигабайт. Задание можно выполнить и на Windows с Python, но я рекомендую поработать под \*NIX-системой (например, через Docker) и активно использовать язык bash.
Немного снобизма (простите, но правда): если вы захотите работать в лучших компаниях мира в области ML, вам все равно понадобится опыт работы с bash под UNIX.

[Веб-форма](https://goo.gl/forms/z8zENbMiaEAeB7nG3) для ответов.

Для выполнения задания понадобится установленный Vowpal Wabbit (уже есть в докер-контейнере курса, см. инструкцию в README [репозитория](https://github.com/Yorko/mlcourse_open) нашего курса) и примерно 70 Гб дискового пространства. Я тестировал решение не на каком-то суперкомпе, а на Macbook Pro 2015 (8 ядер, 16 Гб памяти), и самая тяжеловесная модель обучалась около 12 минут, так что задание реально выполнить и с простым железом. Но если вы планируете когда-либо арендовать сервера Amazon, можно попробовать это сделать уже сейчас.

Материалы в помощь:
 - интерактивный [тьюториал](https://www.codecademy.com/en/courses/learn-the-command-line/lessons/environment/exercises/bash-profile) CodeAcademy по утилитам командной строки UNIX (примерно на час-полтора)
 - [статья](https://habrahabr.ru/post/280562/) про то, как арендовать на Amazon машину (еще раз: это не обязательно для выполнения задания, но будет хорошим опытом, если вы это делаете впервые)

### 2.2. Описание данных

Имеются 10 Гб вопросов со StackOverflow – [скачайте](https://yadi.sk/d/krikdUic3Ggjyf) эту выборку. 

Формат данных простой:<br>
<center>*текст вопроса* (слова через пробел) TAB *теги вопроса* (через пробел)

Здесь TAB – это символ табуляции.
Пример первой записи в выборке:

In [2]:
!head -1 ../../../../datasets/stackoverflow/stackoverflow.10kk.tsv

 is there a way to apply a background color through css at the tr level i can apply it at the td level like this my td background color e8e8e8 background e8e8e8 however the background color doesn t seem to get applied when i attempt to apply the background color at the tr level like this my tr background color e8e8e8 background e8e8e8 is there a css trick to making this work or does css not natively support this for some reason 	css css3 css-selectors


Здесь у нас текст вопроса, затем табуляция и теги вопроса: *css, css3* и *css-selectors*. Всего в выборке таких вопросов 10 миллионов. 

In [3]:
%%time
!wc -l ../../../../datasets/stackoverflow/stackoverflow.10kk.tsv

 10000000 ../../../../datasets/stackoverflow/stackoverflow.10kk.tsv
CPU times: user 5.85 s, sys: 2.15 s, total: 8 s
Wall time: 4min 11s


Обратите внимание на то, что такие данные я уже не хочу загружать в оперативную память и, пока можно, буду пользоваться эффективными утилитами UNIX –  head, tail, wc, cat, cut и прочими.

### 2.3. Предобработка данных

Давайте выберем в наших данных все вопросы с тегами *javascript, java, python, ruby, php, c++, c#, go, scala* и  *swift* и подготовим обучающую выборку в формате Vowpal Wabbit. Будем решать задачу 10-классовой классификации вопросов по перечисленным тегам.

Вообще, как мы видим, у каждого вопроса может быть несколько тегов, но мы упростим себе задачу и будем у каждого вопроса выбирать один из перечисленных тегов либо игнорировать вопрос, если таковых тегов нет. 
Но вообще VW поддерживает multilabel classification (аргумент  --multilabel_oaa).
<br>
<br>
Реализуйте в виде отдельного файла `preprocess.py` код для подготовки данных. Он должен отобрать строки, в которых есть перечисленные теги, и переписать их в отдельный файл в формат Vowpal Wabbit. Детали:
 - скрипт должен работать с аргументами командной строки: с путями к файлам на входе и на выходе
 - строки обрабатываются по одной (можно использовать tqdm для подсчета числа итераций)
 - если табуляций в строке нет или их больше одной, считаем строку поврежденной и пропускаем
 - в противном случае смотрим, сколько в строке тегов из списка *javascript, java, python, ruby, php, c++, c#, go, scala* и  *swift*. Если ровно один, то записываем строку в выходной файл в формате VW: `label | text`, где `label` – число от 1 до 10 (1 - *javascript*, ... 10 – *swift*). Пропускаем те строки, где интересующих тегов больше или меньше одного 
 - из текста вопроса надо выкинуть двоеточия и вертикальные палки, если они есть – в VW это спецсимволы

In [3]:
import os
from tqdm import tqdm
from time import time
import numpy as np
from sklearn.metrics import accuracy_score

Должно получиться вот такое число строк – 4389054. Как видите, 10 Гб у меня обработались примерно за полторы минуты.

In [11]:
!python3 preprocess.py \
    ../../../../datasets/stackoverflow/stackoverflow.10kk.tsv \
    ../../../../datasets/stackoverflow/stackoverflow.vw

10000000it [05:28, 30480.21it/s]
Lines selected: 4389054
Lines corrupted: 15


Поделите выборку на обучающую, проверочную и тестовую части в равной пропорции - по  1463018 в каждый файл. Перемешивать не надо, первые 1463018 строк должны пойти в обучающую часть `stackoverflow_train.vw`, последние 1463018 – в тестовую `stackoverflow_test.vw`, оставшиеся – в проверочную `stackoverflow_valid.vw`. 

Также сохраните векторы ответов для проверочной и тестовой выборки в отдельные файлы `stackoverflow_valid_labels.txt` и `stackoverflow_test_labels.txt`.

Тут вам помогут утилиты `head`, `tail`, `split`, `cat` и `cut`.

In [8]:
#!split -l 1463018 ../../../../datasets/stackoverflow/stackoverflow.vw

### 2.4. Обучение и проверка моделей

Обучите Vowpal Wabbit на выборке `stackoverflow_train.vw` 9 раз, перебирая параметры passes (1,3,5), ngram (1,2,3).
Остальные параметры укажите следующие: bit_precision=28 и seed=17. Также скажите VW, что это 10-классовая задача.

Проверяйте долю правильных ответов на выборке `stackoverflow_valid.vw`. Выберите лучшую модель и проверьте качество на выборке `stackoverflow_test.vw`.

In [29]:
from sklearn.metrics import accuracy_score


with open('../../../../datasets/stackoverflow/stackoverflow_valid.vw') as valid_file:
    y_valid = [int(line.split('|')[0].strip()) for line in valid_file.readlines()]

for passes in [1, 3, 5]:
    for ngram in [1, 2, 3]:
        !vw -c -d ../../../../datasets/stackoverflow/stackoverflow_train.vw -f model_p-{passes}_n-{ngram}.vw \
            --oaa 10 --passes {passes} --ngram {ngram} --bit_precision 28 --random_seed 17 --quiet
        !vw -i model_p-{passes}_n-{ngram}.vw -t -d ../../../../datasets/stackoverflow/stackoverflow_valid.vw \
            -p model_p-{passes}_n-{ngram}_valid_pred.txt --quiet

        with open('model_p-%d_n-%d_valid_pred.txt' % (passes, ngram)) as pred_file:
            valid_prediction = [int(label) for label in pred_file.readlines()]
        print("Accuracy for %d passes and %d ngram: %.6f" % (passes, ngram, accuracy_score(y_valid, valid_prediction)))

Accuracy for 1 passes and 1 ngram: 0.915056
Accuracy for 1 passes and 2 ngram: 0.931078
Accuracy for 1 passes and 3 ngram: 0.928601
Accuracy for 3 passes and 1 ngram: 0.914342
Accuracy for 3 passes and 2 ngram: 0.927766
Accuracy for 3 passes and 3 ngram: 0.926387
Accuracy for 5 passes and 1 ngram: 0.913449
Accuracy for 5 passes and 2 ngram: 0.929261
Accuracy for 5 passes and 3 ngram: 0.926164


<font color='red'> Вопрос 1.</font> Какое сочетание параметров дает наибольшую долю правильных ответов на проверочной выборке `stackoverflow_valid.vw`?
- Биграммы и 3 прохода по выборке
- Триграммы и 5 проходов по выборке
- **Биграммы и 1 проход по выборке**
- Униграммы и 1 проход по выборке

Проверьте лучшую (по доле правильных ответов на валидации) модель на тестовой выборке. 

In [32]:
passes = 1
ngram = 2

with open('../../../../datasets/stackoverflow/stackoverflow_test.vw') as test_file:
    y_test = [int(line.split('|')[0].strip()) for line in test_file.readlines()]

!vw -i model_p-{passes}_n-{ngram}.vw -t -d ../../../../datasets/stackoverflow/stackoverflow_test.vw \
    -p model_p-{passes}_n-{ngram}_test_pred.txt --quiet

with open('model_p-%d_n-%d_test_pred.txt' % (passes, ngram)) as pred_file:
    valid_prediction = [int(label) for label in pred_file.readlines()]
print("Accuracy for %d passes and %d ngram on test dataset: %.6f" % (passes, ngram, accuracy_score(y_test, valid_prediction)))

Accuracy for 1 passes and 2 ngram on test dataset: 0.931191


<font color='red'> Вопрос 2.</font> Как соотносятся доли правильных ответов лучшей (по доле правильных ответов на валидации) модели на проверочной и на тестовой выборках? (здесь % – это процентный пункт, т.е., скажем, снижение с 50% до 40% – это на 10%, а не 20%).
- На тестовой ниже примерно на 2%
- На тестовой ниже примерно на 3%
- **Результаты почти одинаковы – отличаются меньше чем на 0.5%**

Обучите VW с параметрами, подобранными на проверочной выборке, теперь на объединении обучающей и проверочной выборок. Посчитайте долю правильных ответов на тестовой выборке. 

In [34]:
passes = 1
ngram = 2

with open('../../../../datasets/stackoverflow/stackoverflow_test.vw') as test_file:
    y_test = [int(line.split('|')[0].strip()) for line in test_file.readlines()]

!vw -c -d ../../../../datasets/stackoverflow/stackoverflow_train_and_valid.vw \
    -f model_p-{passes}_n-{ngram}_train_and_valid.vw \
    --oaa 10 --passes {passes} --ngram {ngram} --bit_precision 28 --random_seed 17
!vw -i model_p-{passes}_n-{ngram}_train_and_valid.vw -t -d ../../../../datasets/stackoverflow/stackoverflow_test.vw \
    -p model_p-{passes}_n-{ngram}_t_a_v_test_pred.txt --quiet

with open('model_p-%d_n-%d_t_a_v_test_pred.txt' % (passes, ngram)) as pred_file:
    valid_prediction = [int(label) for label in pred_file.readlines()]
print("Accuracy for %d passes and %d ngram on test dataset: %.6f" % (passes, ngram, accuracy_score(y_test, valid_prediction)))

Generating 2-grams for all namespaces.
final_regressor = model_p-1_n-2_train_and_valid.vw
Num weight bits = 28
learning rate = 0.5
initial_t = 0
power_t = 0.5
creating cache_file = ../../../../datasets/stackoverflow/stackoverflow_train_and_valid.vw.cache
Reading datafile = ../../../../datasets/stackoverflow/stackoverflow_train_and_valid.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.000000 0.000000            1            1.0        1        1      320
0.500000 1.000000            2            2.0        4        1      134
0.750000 1.000000            4            4.0        7        1      174
0.750000 0.750000            8            8.0        7        1      188
0.750000 0.750000           16           16.0        7        7      416
0.781250 0.812500           32           32.0        7        2      346
0.765625 0.750000           64           64.0        3    

<font color='red'> Вопрос 3.</font> На сколько процентных пунктов повысилась доля правильных ответов модели после обучения на вдвое большей выборке (обучающая `stackoverflow_train.vw` + проверочная `stackoverflow_valid.vw`) по сравнению с моделью, обученной только на `stackoverflow_train.vw`?
 - 0.1%
 - **0.4%**
 - 0.8%
 - 1.2%

### 2.5. Заключение

В этом задании мы только познакомились с Vowpal Wabbit. Что еще можно попробовать:
 - Классификация с несколькими ответами (multilabel classification, аргумент  `multilabel_oaa`) – формат данных тут как раз под такую задачу
 - Настройка параметров VW с hyperopt, авторы библиотеки утверждают, что качество должно сильно зависеть от параметров изменения шага градиентного спуска (`initial_t` и `power_t`). Также можно потестировать разные функции потерь – обучать логистическую регресиию или линейный SVM
 - Познакомиться с факторизационными машинами и их реализацией в VW (аргумент `lrq`)